# Problem 2
A heatmap of pedestrian and cycling accidents in Finland in 2019.

In [1]:
import pandas as pd
import geopandas as gpd
from pyproj import CRS
from shapely.geometry import Point
import folium
from folium.plugins import HeatMap

In [2]:
# Import the data, select columns that are needed
fp ="data/tieliikenneonnettomuudet_2019_onnettomuus.csv"
# Select the coulmns needed for the map
data = pd.read_csv(fp, delimiter=";", usecols=["Onnett_id", "X", "Y", "Onluokka"])

In [12]:
data.head()

,Onnett_id,Onluokka,X,Y
0,9332527,6,382577.0,6675844.0
1,9343914,1,382543.0,6675979.0
2,9398457,6,382542.0,6675979.0
3,9398382,6,382540.0,6675979.0
4,9533397,1,382491.0,6675945.0


In [4]:
# Select bicycle and pedestrian accidents (Onluokka=8 & 9) and put them into their own dfs

cycling = data.loc[data["Onluokka"]==8]
walking = data.loc[data["Onluokka"]==9]


Turn the data into a geodataframe

In [5]:
# Use function from geopandas for the task
cycling_geo = gpd.GeoDataFrame(
    cycling, geometry=gpd.points_from_xy(cycling.X, cycling.Y), crs="EPSG:3067")

walking_geo = gpd.GeoDataFrame(
    walking, geometry=gpd.points_from_xy(walking.X, walking.Y), crs="EPSG:3067")

In [6]:
cycling_wgs84 = cycling_geo.to_crs(epsg=4326)
walking_wgs84 = walking_geo.to_crs(epsg=4326)

In [7]:
# Convert dataframes to GeoJSON
#cycling_gjson = folium.features.GeoJson(cycling_geo, name="Cycling_accidents")
#walking_gjson = folium.features.GeoJson(walking_geo, name="Pedestrian_accidents")

In [8]:
# Get x and y coordinates for each point
cycling_wgs84["x"] = cycling_wgs84["geometry"].apply(lambda geom: geom.x)
cycling_wgs84["y"] = cycling_wgs84["geometry"].apply(lambda geom: geom.y)

# Create a list of coordinate pairs
cycling_locations = list(zip(cycling_wgs84["y"], cycling_wgs84["x"]))

# Get x and y coordinates for each point
walking_wgs84["x"] = walking_wgs84["geometry"].apply(lambda geom: geom.x)
walking_wgs84["y"] = walking_wgs84["geometry"].apply(lambda geom: geom.y)

# Create a list of coordinate pairs
walking_locations = list(zip(walking_wgs84["y"], walking_wgs84["x"]))

In [11]:
# Create a Map instance
m = folium.Map(location=[63.2, 24.9], 
               zoom_start=6, 
               control_scale=True, 
               tiles = "CartoDB dark_matter",
               name = "Cycling and pedestrian road accidents 2019",
               attribution = "CC-BY 4.0 Väylävirasto", # Couldn't find a way to make this visible :(
               
              )

# Make heatmaps for cycling and walking, add them to their own feature groups and to the map
HeatMap(cycling_locations).add_to(folium.FeatureGroup(name = "Cycling accidents").add_to(m))
# Make the other heatmap off by default to make the map less busy
HeatMap(walking_locations).add_to(folium.FeatureGroup(name = "Pedestrian accidents", show = False).add_to(m))

# Turn layer control on
folium.LayerControl().add_to(m)

# Add map title
# If the map was embedden into a web page, title should be put in the page instead of the map
title_html = '''<h3 align="center" style="font-size:20px"><b>Cycling and pedestrian road accidents 2019</b></h3>'''
m.get_root().html.add_child(folium.Element(title_html))

m

In [10]:
outfp = "accidents_map_2019.html"
m.save(outfp)

The final map doesn't have any fancy data analysis behind it, just finding the right data from the dataset and plotting it. The heatmap-format in itself is a bit restrictive and I couldn't find ways to add information like descrpabout individual 